In [9]:
import pickle
import matplotlib.pyplot as plt
plt.rcParams.update(plt.rcParamsDefault)
import numpy as np
from pygenstability.optimal_scales import identify_optimal_scales
import mat73
import scipy.io
import h5py
import networkx as nx
import networkx as nx
import pygenstability as pgs
import scipy.sparse as sp
import pandas as pd
import seaborn as sns

In [2]:
be_data = scipy.io.loadmat('/Users/sonmjack/Downloads/simon_paper/data_fam1novfam1_trackdata.mat')
mat_trigger = np.load('/Users/sonmjack/Downloads/simon_paper/shengyuan_trigger_fam1.npy')
type_array = h5py.File('/Users/sonmjack/Downloads/simon_paper/data_fam1novfam1_timeseries.mat')

gene = type_array['genotype'][:, :].T
# mat_label = np.zeros((gene.shape[0],4))
# be_phi_sum = be_data['nov_phi']
be_phi_sum_fam1 = be_data['fam1_phi']
be_phi_sum_nov = be_data['nov_phi']
be_phi_sum_fam1r2 = be_data['fam1r2_phi']
be_x = be_data['fam1_x']
be_y = be_data['fam1_y']
be_time = be_data['fam1_time']
be_speed = be_data['fam1_speed']
include_list = []
be_x_list_young = []
be_y_list_young = []
be_time_list_young = []
be_speed_list_young = []
be_phi_list_young_fam1 = []
be_phi_list_young_nov = []
be_phi_list_young_fam1r2 = []
gene_list = []
for i in range(10, 46, 2):  #0, len(mat_trigger), 2
    if i == len(mat_trigger):
        break
    if i == 18:
        pass
    else:
        be_x_list_young.append(be_x[int(i / 2), 0])
        be_y_list_young.append(be_y[int(i / 2), 0])
        be_time_list_young.append(be_time[int(i / 2), 0])
        be_speed_list_young.append(be_speed[int(i / 2), 0])
        be_phi_list_young_fam1.append(be_phi_sum_fam1[int(i / 2), 0])
        be_phi_list_young_nov.append(be_phi_sum_nov[int(i / 2), 0])
        be_phi_list_young_fam1r2.append(be_phi_sum_fam1r2[int(i / 2), 0])
        gene_list.append(mat_trigger[i, 1])

del be_x, be_y, be_time, be_speed, be_phi_sum_fam1,be_phi_sum_nov

with open('/Users/sonmjack/Downloads/age2 result_fam1/fam1_all_Smask.pkl', 'rb') as file:
    mask = pickle.load(file)
    
#env = 'nov'
env = 'fam1'
#env = 'fam1r2'
if env == 'fam1':
    with open('/Users/sonmjack/Downloads/age2 result_fam1/fam1_all_EPSP_young.pkl', 'rb') as file:
        dy_list = pickle.load(file)
    with open('/Users/sonmjack/Downloads/age2 result_fam1/fam1_S_spike.pkl', 'rb') as file:
        neuron_spike = pickle.load(file)
    with open('/Users/sonmjack/Downloads/age2 result_fam1/fam1_signal_corr_WT', 'rb') as file:
        signal_list = pickle.load(file)
    with open('/Users/sonmjack/Downloads/age2 result_fam1/fam1_signal_corr_AD', 'rb') as file:
        signal_list_AD = pickle.load(file)
    with open('/Users/sonmjack/Downloads/age2 result_fam1/fam1_S_tuning curve.pkl', 'rb') as file:
        All_tuning_curve = pickle.load(file)
elif env =="nov":
    with open('/Users/sonmjack/Downloads/age2 result_nov/nov_all_EPSP_young.pkl', 'rb') as file:
        dy_list = pickle.load(file)
    with open('/Users/sonmjack/Downloads/age2 result_nov/nov_S_spike.pkl', 'rb') as file:
        neuron_spike = pickle.load(file)
    with open('/Users/sonmjack/Downloads/age2 result_nov/nov_signal_corr_WT', 'rb') as file:
        signal_list = pickle.load(file)
    with open('/Users/sonmjack/Downloads/age2 result_nov/nov_signal_corr_AD', 'rb') as file:
        signal_list_AD = pickle.load(file)
    with open('/Users/sonmjack/Downloads/age2 result_nov/nov_S_tuning curve.pkl', 'rb') as file:
        All_tuning_curve = pickle.load(file)
elif env=='fam1r2':
    with open('/Users/sonmjack/Downloads/age2 result_fam1r2/fam1r2_all_EPSP_young.pkl', 'rb') as file:
        dy_list = pickle.load(file)
    with open('/Users/sonmjack/Downloads/age2 result_fam1r2/fam1r2_S_spike.pkl', 'rb') as file:
        neuron_spike = pickle.load(file)
    with open('/Users/sonmjack/Downloads/age2 result_fam1r2/fam1r2_signal_corr_WT', 'rb') as file:
        signal_list = pickle.load(file)
    with open('/Users/sonmjack/Downloads/age2 result_fam1r2/fam1r2_signal_corr_AD', 'rb') as file:
        signal_list_AD = pickle.load(file)
    with open('/Users/sonmjack/Downloads/age2 result_fam1r2/fam1r2_S_tuning curve.pkl', 'rb') as file:
        All_tuning_curve = pickle.load(file)

In [38]:
plt.figure(figsize=(20, 6))
plt.plot(be_phi_list_young_fam1[5][0:40000])
plt.title('Fam1')
plt.figure(figsize=(20, 6))
plt.plot(be_phi_list_young_nov[5][0:40000])
plt.title('Nov')
plt.figure(figsize=(20, 6))
plt.plot(be_phi_list_young_fam1r2[5][0:40000])
plt.title('Fam1r2')

Text(0.5, 1.0, 'Fam1r2')

In [39]:
outlier_ratio_list_fam1 = []
# outlier_ratio_list_nov = []
# outlier_ratio_list_nov1 = []
#utlier_ratio_list_fam1r2 = []


for i in range(len(be_phi_list_young_fam1)):
    if gene_list[i] == 119:
        angle_fam1 = be_phi_list_young_fam1[i].reshape(-1)
        angle_nov = be_phi_list_young_nov[i].reshape(-1)
        angle_fam1r2 = be_phi_list_young_fam1r2[i].reshape(-1)
        bins = np.arange(0, 370, 10)  # 创建从0到360度，步长为10的区间
        
        angle_categories = pd.cut(angle_nov, bins, right=False, labels=[f'{i}°-{i+10}°' for i in range(0, 360, 10)])
    
        angle_distribution = pd.value_counts(angle_categories, normalize=True).sort_index()
        
        # 将结果转换为DataFrame以便使用Seaborn进行绘图
        df = pd.DataFrame({'Angle Range': angle_distribution.index, 'Percentage': angle_distribution.values})

        observed_frequencies = pd.value_counts(angle_categories, sort=False).sort_index()
        
        # 理论上的均匀分布频数（每个区间的预期频数）
        expected_frequencies = np.full_like(observed_frequencies, fill_value=len(angle_nov) / len(bins[:-1]))
        
        # 计算标准误差（在均匀分布的假设下，使用泊松分布的近似，标准误差为理论频数的平方根）
        std_error = np.sqrt(expected_frequencies)
        
        # 计算每个区间的Z-score
        z_scores = (observed_frequencies - expected_frequencies) / std_error
        
        # 输出结果
        # print("Z-scores for each 10° interval:")
        # print(z_scores)        
        # 找出异常大的Z-score值，通常|Z| > 2 被认为是显著的
        significant_intervals = z_scores[z_scores > 30]
        
        
        precent = np.sum(observed_frequencies[significant_intervals.index].values)/np.sum(observed_frequencies[:].values)
        # print("Number of outliers:", outlier_count)
        # print("Outliers: ", outlier_values)
        # print("Outlier ratio:", outlier_ratio)
        # outlier_ratio_list_fam1.append(precent)
        # outlier_ratio_list_fam1r2.append(precent)
        # outlier_ratio_list_nov1.append(significant_intervals.size)
        # outlier_ratio_list_nov.append(precent)
 
 
        # plt.figure(figsize=(14, 7))
        # sns.barplot(x='Angle Range', y='Percentage', data=df)
        # plt.xticks(rotation=90)  # 旋转x轴标签以便更容易阅读
        # plt.title('Distribution of Angles of'+str(i))
        # plt.ylabel('Percentage')
        # plt.xlabel('Angle Interval')
        # plt.show()

## Community based on signal connectivity

In [3]:

def Connector(Q):
    D = nx.to_networkx_graph(Q, create_using=nx.DiGraph())
    Isolate_list = list(nx.isolates(D))
    if len(Isolate_list) > 0:
        for i in Isolate_list:
            if i == 0:
                Q[i + 1, i] = 0.0001
            else:
                Q[i - 1, i] = 0.0001
    del D
    return Q
def normal(A):
    np.fill_diagonal(A, 0)
    min_val = np.min(A)
    max_val = np.max(A)
    A = (A - min_val) / (max_val - min_val)

    return A

def sparse (A):
    N = A.shape[0]
    # print(max(A[:,4]))
    # A=np.where(A > 0.09, 1, 0)
    k = int(N/2)

    # W.sort(reverse=True)
    B1 = np.zeros((N, N))
    for i in range(N):
        W = sorted(A[i, :], reverse=True)
        #     print( W[k])
        B1[i, :] = np.where(A[i, :] > W[k], 1, 0)

    # B=np.multiply(B1,A)
    # print(W[k])
    # print(A[20,1:20])
    # print(B[20,1:20])

    C1 = np.zeros((N, N))


    for i in range(N):
        W = sorted(A[:, i], reverse=True)
        #     print( W[k])
        C1[:, i] = np.where(A[:, i] > W[k], 1, 0)
    # C=np.multiply(C1,A)
    Q1 = B1 + C1
    Q2 = np.where(Q1 > .9, 1, 0)

    Q = np.multiply(Q2, A)


    # del A
    for i in range(Q.shape[0]):
        # 检查该行是否全为零
        if np.all(Q[i] == 0):
            # 如果是全为零，随机选择一个元素，并将其赋值为 0.001
            #random_index = np.random.randint(0, Q.shape[1]-1)-1  # 随机选择一个列索引
            Q[i, i-1] = 0.001
            Q[i-1,i] = 0.001
    return Q

import plotly.graph_objects as go  # pragma: no cover
from plotly.offline import plot

def plot_sankey(
    all_results,
    optimal_scales=True,
    live=False,
    filename='',
    scale_index=None,
):  # pragma: no cover
    """Plot Sankey diagram of communities accros scale (plotly only).
    Args:
        all_results (dict): results from run function
        optimal_scales (bool): use optimal scales or not
        live (bool): if True, interactive figure will appear in browser
        filename (str): filename to save the plot
        scale_index (bool): plot scale of indices
    """
    sources = []
    targets = []
    values = []
    shift = 0

    # if not scale_index:
    all_results["community_id_reduced"] = all_results["community_id"][0::10]

    community_ids = all_results["community_id_reduced"]

    for i in range(len(community_ids) - 1):
        community_source = np.array(community_ids[i])
        community_target = np.array(community_ids[i + 1])
        source_ids = set(community_source)
        target_ids = set(community_target)
#         print(target_ids)
        for source in source_ids:
            for target in target_ids:
                value = sum(community_target[community_source == source] == target)
#                 print(community_target[community_source == source] == target)
                if value > 0:
                    values.append(value)
                    sources.append(source + shift)
                    targets.append(target + len(source_ids) + shift)
        shift += len(source_ids)

    fig = go.Figure(
        data=[
            go.Sankey(
                node={
                    "pad": 1,
                    "thickness": 1,
                    "line": {"color": "black", "width": 0.0},
                },
                link={"source": sources, "target": targets, "value": values},
            )
        ],
    )

    fig.update_layout(showlegend=False)
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)
    fig.update_layout(
    paper_bgcolor='white',  # 设置图表的背景颜色为白色
    plot_bgcolor='white')
    
    Scale_enumerate=[r'$S_{%s}$' % (len(community_ids)-i)  for i in range(len(community_ids))]
    for x_coordinate, column_name in enumerate(Scale_enumerate):
        fig.add_annotation(
              x=x_coordinate,#Plotly recognizes 0-5 to be the x range.

              y=1.075,#y value above 1 means above all nodes
              xref="x",
              yref="paper",
              text=column_name,#Text
              showarrow=False,
              font=dict(
                  family="Tahoma",
                  size=16,
                  color="black"
                  ),
              align="left",
              )
    fig.write_image(filename)

In [ ]:
for index in range(len(signal_list_AD)):
    type = 'wild type'
    #Signal= signal_list[index]
    Signal = signal_list_AD[index]
    Signal[Signal <= 0] = 0
    Signal[np.isinf(Signal)] = 0
    Signal = np.nan_to_num(Signal, nan=0)
    
    Q = sparse(Signal)
    D = nx.to_networkx_graph(Q)
    adjacency = nx.adjacency_matrix(D)
    all_results = pgs.run(adjacency,
                          min_scale=-1.5,
                          max_scale=1.5,
                          n_scale=200,
                          n_tries = 600,
                          constructor="continuous_normalized")
          
    # if env == 'fam1':
    #     with open('/Users/sonmjack/Downloads/age2 result_fam1/fam1_Signal_Markov_'+ str(index) +'.pkl', 'wb') as f:
    #         pickle.dump(all_results, f)
    # elif env =="nov":
    #     with open('/Users/sonmjack/Downloads/age2 result_nov/nov_Signal_Markov_'+ str(index) +'.pkl', 'wb') as f:
    #         pickle.dump(all_results, f)
    # elif env=='fam1r2':
    #     with open('/Users/sonmjack/Downloads/age2 result_fam1r2/fam1r2_Signal_Markov_'+ str(index) +'.pkl', 'wb') as f:
    #         pickle.dump(all_results, f)
            
    if env == 'fam1':
        with open('/Users/sonmjack/Downloads/age2 result_fam1/fam1_Signal_Markov_AD'+ str(index) +'.pkl', 'wb') as f:
            pickle.dump(all_results, f)
    elif env =="nov":
        with open('/Users/sonmjack/Downloads/age2 result_nov/nov_Signal_Markov_AD'+ str(index) +'.pkl', 'wb') as f:
            pickle.dump(all_results, f)
    elif env=='fam1r2':
        with open('/Users/sonmjack/Downloads/age2 result_fam1r2/fam1r2_Signal_Markov_AD'+ str(index) +'.pkl', 'wb') as f:
            pickle.dump(all_results, f)

## (Signal) Analysis the community and the mask of neurons

In [13]:
import numpy as np
import matplotlib.colors as mcolors
import random

cluster_number = []

Color_Code2 = ['white','darkblue','red', "orange", "pink", "olive", "cyan","yellow", "green", "brown", "gray", "tomato",'limegreen']

list_color = []
for color in mcolors.CSS4_COLORS:
    list_color.append(color)
random.shuffle(list_color)
Color_Code1 = [*Color_Code2, *list_color]
Color_Code = [*Color_Code1, *list_color]
z = 0
v = 0
for index in range(len(dy_list)):#len(dy_list)
    plt.close()
    if gene_list[index] == 119:
        Spike_train = neuron_spike[index][:,int(neuron_spike[index].shape[1]/3):int(neuron_spike[index].shape[1]*2/3)]
        mask_index = mask[index]
        tuning_curve = All_tuning_curve[index]
        type = 'wild type'
        if env == 'fam1':
            with open('/Users/sonmjack/Downloads/age2 result_fam1/fam1_Signal_Markov_'+ str(z) +'.pkl', 'rb') as file:
                all_results = pickle.load(file)
            all_results = identify_optimal_scales(all_results,kernel_size=6,window_size=6)
            _ = pgs.plot_scan(all_results)
            plt.tight_layout()
            plt.savefig('/Users/sonmjack/Downloads/age2 result_fam1/Signal_Markov_fig/'+str(index)+'_young_fam1_markov.pdf')
            plt.close()

        elif env =="nov":
            with open('/Users/sonmjack/Downloads/age2 result_nov/nov_Signal_Markov_'+ str(z) +'.pkl', 'rb') as file:
                all_results = pickle.load(file)
            all_results = identify_optimal_scales(all_results,kernel_size=6,window_size=6)
            _ = pgs.plot_scan(all_results)
            plt.tight_layout()
            plt.savefig('/Users/sonmjack/Downloads/age2 result_nov/Signal_Markov_fig/'+str(index)+'_young_nov_markov.pdf')
            plt.close()
        elif env =="fam1r2":
            with open('/Users/sonmjack/Downloads/age2 result_fam1r2/fam1r2_Signal_Markov_'+ str(z) +'.pkl', 'rb') as file:
                all_results = pickle.load(file)
            all_results = identify_optimal_scales(all_results,kernel_size=6,window_size=6)
            _ = pgs.plot_scan(all_results)
            plt.tight_layout()
            plt.savefig('/Users/sonmjack/Downloads/age2 result_fam1r2/Signal_Markov_fig/'+str(index)+'_young_fam1r2_markov.pdf')
            plt.close()

        
        if env == 'fam1'or env=='fam1r2':
            closest_number = min(all_results['selected_partitions'], key=lambda x: abs(x - 133))
            Community = all_results['community_id'][closest_number]
            
        elif env =="nov" :
            selected_partitions = all_results['selected_partitions']
            Community = all_results['community_id'][199]
            # if selected_partitions == []:
            #     Community = all_results['community_id'][199]
            # else:
            #     closest_number = min(all_results['selected_partitions'], key=lambda x: abs(x - 133))
            #     Community = all_results['community_id'][closest_number]
 
        non_zero_indices_per_row = []
        for row in Spike_train:
            # 找到每行中不为0的元素的列索引
            non_zero_indices = np.where(row != 0)[0]
            # 添加到列表中
            non_zero_indices_per_row.append(list(non_zero_indices))
        neuralData = []
        ColorCode = []
        C_tuning_curve = []
        it = 0
        

        for i in range(max(Community) + 1):
            for m in range(len(non_zero_indices_per_row)):
                #这里最好再分一些小列表，然后根据tuning curve进行划分
                if Community[m] == i:
                    C_tuning_curve.append(tuning_curve[m])
                if m == len(non_zero_indices_per_row)-1:
                    test= np.full(tuning_curve[m].shape[0], np.nan)
                    C_tuning_curve.append(np.full(tuning_curve[m].shape[0], np.nan))
                    C_tuning_curve.append(np.full(tuning_curve[m].shape[0], np.nan))
                    C_tuning_curve.append(np.full(tuning_curve[m].shape[0], np.nan))
                    
        C_tuning_curve = np.array(C_tuning_curve)
        
        plt.close()
        fig, axes = plt.subplots(2, 1, figsize=(15, 12), gridspec_kw={'height_ratios': [6, 3]})
        sns.heatmap(C_tuning_curve, cmap='viridis',vmax=1,cbar=False,ax=axes[0])
        
        if env == 'fam1':
            angle_fam1 = be_phi_list_young_fam1[index].reshape(-1)
            bins = np.linspace(0, 360, num=101)
            even_labels = [str(i) for i in range(0, 100, 2)]
            angle_categories = pd.cut(angle_fam1, bins, right=False,labels=[str(i) for i in range(100)])
            angle_distribution = pd.value_counts(angle_categories, normalize=True).sort_index()

            df = pd.DataFrame({'Angle Range': angle_distribution.index, 'Percentage': angle_distribution.values})
            observed_frequencies = pd.value_counts(angle_categories, sort=False).sort_index()
            sns.barplot(x='Angle Range', y='Percentage', data=df,ax =axes[1])
            x_labels = [label if int(label) % 2 == 0 else '' for label in df['Angle Range']]
            # axes[1].set_xticklabels(x_labels, rotation=90)
            axes[1].set_xticklabels(x_labels)
            #axes[1].set_title('Distribution of Angles of '+str(index),fontsize=20)
            axes[1].set_ylabel('Percentage',fontsize=20)
            axes[1].set_xlabel('Position (cm)',fontsize=20)
            plt.savefig('/Users/sonmjack/Downloads/age2 result_fam1/Signal_Markov_fig/'+str(index)+'_young_fam1_tuning.pdf')
            plt.close()    
        elif env =="nov":     
            angle_nov = be_phi_list_young_nov[index].reshape(-1)
            bins = np.linspace(0, 360, num=101)
            even_labels = [str(i) for i in range(0, 100, 2)]
            angle_categories = pd.cut(angle_nov, bins, right=False,labels=[str(i) for i in range(100)])
            angle_distribution = pd.value_counts(angle_categories, normalize=True).sort_index()

            df = pd.DataFrame({'Angle Range': angle_distribution.index, 'Percentage': angle_distribution.values})
            observed_frequencies = pd.value_counts(angle_categories, sort=False).sort_index()
            sns.barplot(x='Angle Range', y='Percentage', data=df,ax =axes[1])
            x_labels = [label if int(label) % 2 == 0 else '' for label in df['Angle Range']]
            # axes[1].set_xticklabels(x_labels, rotation=90)
            axes[1].set_xticklabels(x_labels)
            #axes[1].set_title('Distribution of Angles of '+str(index),fontsize=20)
            axes[1].set_ylabel('Percentage',fontsize=20)
            axes[1].set_xlabel('Position (cm)',fontsize=20)
            plt.savefig('/Users/sonmjack/Downloads/age2 result_nov/Signal_Markov_fig/'+str(index)+'_young_nov_tuning.pdf')
            plt.close()
        elif env =="fam1r2":
            angle_fam1r2 = be_phi_list_young_fam1r2[index].reshape(-1)
            bins = np.linspace(0, 360, num=101)
            even_labels = [str(i) for i in range(0, 100, 2)]
            angle_categories = pd.cut(angle_fam1r2, bins, right=False,labels=[str(i) for i in range(100)])
            angle_distribution = pd.value_counts(angle_categories, normalize=True).sort_index()

            df = pd.DataFrame({'Angle Range': angle_distribution.index, 'Percentage': angle_distribution.values})
            observed_frequencies = pd.value_counts(angle_categories, sort=False).sort_index()
            sns.barplot(x='Angle Range', y='Percentage', data=df,ax =axes[1])
            x_labels = [label if int(label) % 2 == 0 else '' for label in df['Angle Range']]
            # axes[1].set_xticklabels(x_labels, rotation=90)
            axes[1].set_xticklabels(x_labels,rotation=0)
            #axes[1].set_title('Distribution of Angles of '+str(index),fontsize=20)
            axes[1].set_ylabel('Percentage',fontsize=20)
            axes[1].set_xlabel('Position (cm)',fontsize=20)
            plt.savefig('/Users/sonmjack/Downloads/age2 result_fam1r2/Signal_Markov_fig/'+str(index)+'_young_fam1r2_tuning.pdf')
            plt.close()
        
        
        Nodes = range(len(non_zero_indices_per_row))
        for m, j in zip(Nodes, Nodes):
            for i in range(max(Community) + 1):
                if Community[j] == i:
                    non_zero_indices_per_row[int(m)].sort()
                    neuralData.append(non_zero_indices_per_row[int(m)])
                    ColorCode.append(Color_Code[i+1])
                    # A_ordered_row[it,:]=Q[j,:]
                    it += 1
        # it=0
        # for i in range(max(Community)+1):
        #     for m,j in zip(Nodes,range(N)):
        #         if Community[j]==i:
        #             A_ordered[:,it]=A_ordered_row[:,j]
        #             it+=1
        fig, axes = plt.subplots(2, 1, figsize=(27, 9), gridspec_kw={'height_ratios': [6, 3]})
        axes[0].eventplot(neuralData, color=ColorCode)
        axes[0].invert_yaxis()

        axes[0].set_title("Reordered neural spike trains " + f'N-{max(Community) + 1}',fontsize=20)
        axes[0].set_ylabel('Rearranged Neuron ID', fontsize=20)
        
        # max_in_sublists = [max(sublist) for sublist in neuralData if sublist]
        # 
        # # 找到最大值中的最大值
        # overall_max = max(max_in_sublists)
        # nearest_5000_multiple = round(overall_max / 10000) * 10000
        # axes[0].set_xticks(np.arange(0,nearest_5000_multiple, 10000), [str(int(i*10000/30)) for i in range(0, round(overall_max / 10000))],fontsize=13)
        #plt.show()

        if env == 'fam1':
            axes[1].plot(be_phi_list_young_fam1[index][int(neuron_spike[index].shape[1]/3):int(neuron_spike[index].shape[1]*2/3)])
            #axes[1].set_xticks(np.arange(0,nearest_5000_multiple, 10000), [str(int(i*10000/30)) for i in range(0, round(overall_max / 10000))],fontsize=13)         
            axes[1].set_xlabel('Time', fontsize=20)
            axes[1].set_ylabel('Position°', fontsize=20)
            axes[1].invert_yaxis()
            plt.savefig('/Users/sonmjack/Downloads/age2 result_fam1/Signal_Markov_fig/'+str(index)+'_young_fam1_markov_neuron_list.pdf')
            plt.close()    
        elif env =="nov":
            axes[1].plot(be_phi_list_young_nov[index][int(neuron_spike[index].shape[1]/3):int(neuron_spike[index].shape[1]*2/3)])
            #axes[1].set_xticks(np.arange(0,nearest_5000_multiple, 10000), [str(int(i*10000/30)) for i in range(0, round(overall_max / 10000))],fontsize=13)  
            axes[1].set_xlabel('Time', fontsize=20)
            axes[1].set_ylabel('Position°', fontsize=20)
            axes[1].invert_yaxis()
            plt.savefig('/Users/sonmjack/Downloads/age2 result_nov/Signal_Markov_fig/'+str(index)+'_young_nov_markov_neuron_list.pdf')
            plt.close()
        elif env =="fam1r2":
            axes[1].plot(be_phi_list_young_fam1r2[index][int(neuron_spike[index].shape[1]/3):int(neuron_spike[index].shape[1]*2/3)])
            #axes[1].set_xticks(np.arange(0,nearest_5000_multiple, 10000), [str(int(i*10000/30)) for i in range(0, round(overall_max / 10000))],fontsize=13) 
            axes[1].set_xlabel('Time', fontsize=20)
            axes[1].set_ylabel('Position°', fontsize=20)
            axes[1].invert_yaxis()
            plt.savefig('/Users/sonmjack/Downloads/age2 result_fam1r2/Signal_Markov_fig/neuron_list_'+str(index)+'_young_fam1r2_markov.pdf')
            plt.close()
            
        from matplotlib.colors import BoundaryNorm
        
        pic = np.zeros((512, 512))
        for i in range(mask_index.shape[2]):
            index_row = np.where(mask_index[:, :, i] == True)[0]
            index_con = np.where(mask_index[:, :, i] == True)[1]
            for j in range(len(index_row)):
                    pic[index_row[j], index_con[j]] = Community[i]+1
        #cmap = sns.diverging_palette(220, 20, as_cmap=True)
        import seaborn as sns
        
        cmap = plt.get_cmap('tab10')  

        from matplotlib.colors import LinearSegmentedColormap
        cmap = LinearSegmentedColormap.from_list("custom_cmap", Color_Code2, N=len(Color_Code2))
        bounds = np.linspace(-0.5, 11.5, 13)  
        norm = BoundaryNorm(bounds, cmap.N)
        sns.heatmap(pic, cmap=cmap, norm=norm)
        plt.xticks([])
        plt.yticks([])
        plt.title(f'Scale= {closest_number*1.5/200:.2f}, N='+str(max(Community) + 1),fontsize=20)
        plt.tight_layout()
        # plt.title('Community beasd on real data')
        #plt.show()
        if env == 'fam1':
            plt.savefig('/Users/sonmjack/Downloads/age2 result_fam1/Signal_Markov_fig/'+str(index)+'_young_fam1_markov_neuron_mask.pdf')
            plt.close()

        elif env =="nov":
            plt.savefig('/Users/sonmjack/Downloads/age2 result_nov/Signal_Markov_fig/'+str(index)+'_young_nov_markov_neuron_mask.pdf')
            plt.close()
        elif env =="fam1r2":
            plt.savefig('/Users/sonmjack/Downloads/age2 result_fam1r2/Signal_Markov_fig/mask_'+str(index)+'_young_fam1r2_markov_neuron.pdf')
            plt.close()            

        
        if env == 'fam1':
            name = '/Users/sonmjack/Downloads/age2 result_fam1/Signal_Markov_fig/'+str(index)+'_young_fam1_sankey.png'   
        elif env =="nov":
            name = '/Users/sonmjack/Downloads/age2 result_nov/Signal_Markov_fig/'+str(index)+'_young_nov_snakey.png'
        elif env =="fam1r2":
            name = '/Users/sonmjack/Downloads/age2 result_fam1r2/Signal_Markov_fig/snakey'+str(index)+'_young_fam1r2.png'
        
            
        plot_sankey(
            all_results,
            optimal_scales=True,
            live=False,
            filename=name ,
            scale_index=None,
        )

        z = z+1
    else:
        Spike_train = neuron_spike[index][:,int(neuron_spike[index].shape[1]/3):int(neuron_spike[index].shape[1]*2/3)]
        mask_index = mask[index]
        tuning_curve = All_tuning_curve[index]
        type = 'wild type'
        if env == 'fam1':
            with open('/Users/sonmjack/Downloads/age2 result_fam1/fam1_Signal_Markov_AD'+ str(v) +'.pkl', 'rb') as file:
                all_results = pickle.load(file)
            all_results = identify_optimal_scales(all_results,kernel_size=6,window_size=6)
            _ = pgs.plot_scan(all_results)
            plt.tight_layout()
            plt.savefig('/Users/sonmjack/Downloads/age2 result_fam1/Signal_Markov_fig_AD/'+str(index)+'_young_fam1_markov.pdf')
            plt.close()

        elif env =="nov":
            with open('/Users/sonmjack/Downloads/age2 result_nov/nov_Signal_Markov_AD'+ str(v) +'.pkl', 'rb') as file:
                all_results = pickle.load(file)
            all_results = identify_optimal_scales(all_results,kernel_size=6,window_size=6)
            _ = pgs.plot_scan(all_results)
            plt.tight_layout()
            plt.savefig('/Users/sonmjack/Downloads/age2 result_nov/Signal_Markov_fig_AD/'+str(index)+'_young_nov_markov.pdf')
            plt.close()
        elif env =="fam1r2":
            with open('/Users/sonmjack/Downloads/age2 result_fam1r2/fam1r2_Signal_Markov_AD'+ str(v) +'.pkl', 'rb') as file:
                all_results = pickle.load(file)
            all_results = identify_optimal_scales(all_results,kernel_size=6,window_size=6)
            _ = pgs.plot_scan(all_results)
            plt.tight_layout()
            plt.savefig('/Users/sonmjack/Downloads/age2 result_fam1r2/Signal_Markov_fig_AD/'+str(index)+'_young_fam1r2_markov.pdf')
            plt.close()

    
        if env == 'fam1'or env=='fam1r2':
            closest_number = min(all_results['selected_partitions'], key=lambda x: abs(x - 133))
            Community = all_results['community_id'][closest_number]
            
        elif env =="nov" :
            selected_partitions = all_results['selected_partitions']
            Community = all_results['community_id'][199]
            # if selected_partitions == []:
            #     Community = all_results['community_id'][199]
            # else:
            #     closest_number = min(all_results['selected_partitions'], key=lambda x: abs(x - 133))
            #     Community = all_results['community_id'][closest_number]
 
        non_zero_indices_per_row = []
        for row in Spike_train:
            # 找到每行中不为0的元素的列索引
            non_zero_indices = np.where(row != 0)[0]
            # 添加到列表中
            non_zero_indices_per_row.append(list(non_zero_indices))
        neuralData = []
        ColorCode = []
        C_tuning_curve = []
        it = 0
        
        
        for i in range(max(Community) + 1):
            for m in range(len(non_zero_indices_per_row)):
                #这里最好再分一些小列表，然后根据tuning curve进行划分
                if Community[m] == i:
                    C_tuning_curve.append(tuning_curve[m])
                if m == len(non_zero_indices_per_row)-1:
                    C_tuning_curve.append(np.full(tuning_curve[m].shape[0], np.nan))
                    C_tuning_curve.append(np.full(tuning_curve[m].shape[0], np.nan))
                    C_tuning_curve.append(np.full(tuning_curve[m].shape[0], np.nan))
                    
        C_tuning_curve = np.array(C_tuning_curve)
        plt.close()
        
        fig, axes = plt.subplots(2, 1, figsize=(15, 12), gridspec_kw={'height_ratios': [6, 3]})
        sns.heatmap(C_tuning_curve, cmap='viridis',vmax=1,cbar=False,ax=axes[0])
        axes[0].set_ylabel('Rearranged Neuron ID',fontsize=20)
        if env == 'fam1':
            angle_fam1 = be_phi_list_young_fam1[index].reshape(-1)
            bins = np.linspace(0, 360, num=101)
            even_labels = [str(i) for i in range(0, 100, 2)]
            angle_categories = pd.cut(angle_fam1, bins, right=False,labels=[str(i) for i in range(100)])
            angle_distribution = pd.value_counts(angle_categories, normalize=True).sort_index()

            df = pd.DataFrame({'Angle Range': angle_distribution.index, 'Percentage': angle_distribution.values})
            observed_frequencies = pd.value_counts(angle_categories, sort=False).sort_index()
            sns.barplot(x='Angle Range', y='Percentage', data=df,ax =axes[1])
            x_labels = [label if int(label) % 2 == 0 else '' for label in df['Angle Range']]
            # axes[1].set_xticklabels(x_labels, rotation=90)
            axes[1].set_xticklabels(x_labels)
            #axes[1].set_title('Distribution of Angles of '+str(index),fontsize=20)
            axes[1].set_ylabel('Percentage',fontsize=20)
            axes[1].set_xlabel('Position (cm)',fontsize=20)
            plt.tight_layout()
            plt.savefig('/Users/sonmjack/Downloads/age2 result_fam1/Signal_Markov_fig_AD/'+str(index)+'_young_fam1_tuning.pdf')
            plt.close()    
        elif env =="nov":     
            angle_nov = be_phi_list_young_nov[index].reshape(-1)
            bins = np.linspace(0, 360, num=101)
            even_labels = [str(i) for i in range(0, 100, 2)]
            angle_categories = pd.cut(angle_nov, bins, right=False,labels=[str(i) for i in range(100)])
            angle_distribution = pd.value_counts(angle_categories, normalize=True).sort_index()

            df = pd.DataFrame({'Angle Range': angle_distribution.index, 'Percentage': angle_distribution.values})
            observed_frequencies = pd.value_counts(angle_categories, sort=False).sort_index()
            sns.barplot(x='Angle Range', y='Percentage', data=df,ax =axes[1])
            x_labels = [label if int(label) % 2 == 0 else '' for label in df['Angle Range']]
            # axes[1].set_xticklabels(x_labels, rotation=90)
            axes[1].set_xticklabels(x_labels)
            #axes[1].set_title('Distribution of Angles of '+str(index))
            axes[1].set_ylabel('Percentage',fontsize=20)
            axes[1].set_xlabel('Position (cm)',fontsize=20)
            plt.tight_layout()
            plt.savefig('/Users/sonmjack/Downloads/age2 result_nov/Signal_Markov_fig_AD/'+str(index)+'_young_nov_tuning.pdf')
            plt.close()
        elif env =="fam1r2":
            angle_fam1r2 = be_phi_list_young_fam1r2[index].reshape(-1)
            bins = np.linspace(0, 360, num=101)
            even_labels = [str(i) for i in range(0, 100, 2)]
            angle_categories = pd.cut(angle_fam1r2, bins, right=False,labels=[str(i) for i in range(100)])
            angle_distribution = pd.value_counts(angle_categories, normalize=True).sort_index()

            df = pd.DataFrame({'Angle Range': angle_distribution.index, 'Percentage': angle_distribution.values})
            observed_frequencies = pd.value_counts(angle_categories, sort=False).sort_index()
            sns.barplot(x='Angle Range', y='Percentage', data=df,ax =axes[1])
            x_labels = [label if int(label) % 2 == 0 else '' for label in df['Angle Range']]
            # axes[1].set_xticklabels(x_labels, rotation=90)
            axes[1].set_xticklabels(x_labels,rotation=0)
            #axes[1].set_title('Distribution of Angles of '+str(index))
            axes[1].set_ylabel('Percentage',fontsize=20)
            axes[1].set_xlabel('Position (cm)',fontsize=20)
            plt.tight_layout()
            plt.savefig('/Users/sonmjack/Downloads/age2 result_fam1r2/Signal_Markov_fig_AD/'+str(index)+'_young_fam1r2_tuning.pdf')
            plt.close()
        
        
        Nodes = range(len(non_zero_indices_per_row))
        for m, j in zip(Nodes, Nodes):
            for i in range(max(Community) + 1):
                if Community[j] == i:
                    non_zero_indices_per_row[int(m)].sort()
                    neuralData.append(non_zero_indices_per_row[int(m)])
                    ColorCode.append(Color_Code[i+1])
                    # A_ordered_row[it,:]=Q[j,:]
                    it += 1
        # it=0
        # for i in range(max(Community)+1):
        #     for m,j in zip(Nodes,range(N)):
        #         if Community[j]==i:
        #             A_ordered[:,it]=A_ordered_row[:,j]
        #             it+=1
        fig, axes = plt.subplots(2, 1, figsize=(27, 9), gridspec_kw={'height_ratios': [6, 3]})
        axes[0].eventplot(neuralData, color=ColorCode)
        axes[0].invert_yaxis()

        axes[0].set_title("Reordered neural spike trains " + f'N-{max(Community) + 1}',fontsize=20)
        axes[0].set_ylabel('Rearranged Neuron ID', fontsize=20)
        
        # max_in_sublists = [max(sublist) for sublist in neuralData if sublist]
        # 
        # # 找到最大值中的最大值
        # overall_max = max(max_in_sublists)
        # nearest_5000_multiple = round(overall_max / 10000) * 10000
        # axes[0].set_xticks(np.arange(0,nearest_5000_multiple, 10000), [str(int(i*10000/30)) for i in range(0, round(overall_max / 10000))],fontsize=13)
        #plt.show()

        if env == 'fam1':
            axes[1].plot(be_phi_list_young_fam1[index][int(neuron_spike[index].shape[1]/3):int(neuron_spike[index].shape[1]*2/3)])
            #axes[1].set_xticks(np.arange(0,nearest_5000_multiple, 10000), [str(int(i*10000/30)) for i in range(0, round(overall_max / 10000))],fontsize=13)  
            axes[1].set_xlabel('Time', fontsize=20)
            axes[1].set_ylabel('Position°', fontsize=20)
            axes[1].invert_yaxis()
            plt.tight_layout()
            plt.savefig('/Users/sonmjack/Downloads/age2 result_fam1/Signal_Markov_fig_AD/'+str(index)+'_young_fam1_markov_neuron_list.pdf')
            plt.close()    
        elif env =="nov":
            axes[1].plot(be_phi_list_young_nov[index][int(neuron_spike[index].shape[1]/3):int(neuron_spike[index].shape[1]*2/3)])
            #axes[1].set_xticks(np.arange(0,nearest_5000_multiple, 10000), [str(int(i*10000/30)) for i in range(0, round(overall_max / 10000))],fontsize=13)  
            axes[1].invert_yaxis()
            axes[1].set_xlabel('Time', fontsize=20)
            axes[1].set_ylabel('Position°', fontsize=20)
            plt.tight_layout()
            plt.savefig('/Users/sonmjack/Downloads/age2 result_nov/Signal_Markov_fig_AD/'+str(index)+'_young_nov_markov_neuron_list.pdf')
            plt.close()
        elif env =="fam1r2":
            axes[1].plot(be_phi_list_young_fam1r2[index][int(neuron_spike[index].shape[1]/3):int(neuron_spike[index].shape[1]*2/3)])
            #axes[1].set_xticks(np.arange(0,nearest_5000_multiple, 10000), [str(int(i*10000/30)) for i in range(0, round(overall_max / 10000))],fontsize=13) 
            axes[1].invert_yaxis()
            axes[1].set_xlabel('Time', fontsize=20)
            axes[1].set_ylabel('Position°', fontsize=20)
            plt.tight_layout()
            plt.savefig('/Users/sonmjack/Downloads/age2 gloryresult_fam1r2/Signal_Markov_fig_AD/neuron_list_'+str(index)+'_young_fam1r2_markov.pdf')
            plt.close()
            
        from matplotlib.colors import BoundaryNorm
        
        pic = np.zeros((512, 512))
        for i in range(mask_index.shape[2]):
            index_row = np.where(mask_index[:, :, i] == True)[0]
            index_con = np.where(mask_index[:, :, i] == True)[1]
            for j in range(len(index_row)):
                    pic[index_row[j], index_con[j]] = Community[i]+1
        #cmap = sns.diverging_palette(220, 20, as_cmap=True)
        import seaborn as sns
        
        cmap = plt.get_cmap('tab10')  

        from matplotlib.colors import LinearSegmentedColormap
        cmap = LinearSegmentedColormap.from_list("custom_cmap", Color_Code2, N=len(Color_Code2))
        bounds = np.linspace(-0.5, 11.5, 13)  
        norm = BoundaryNorm(bounds, cmap.N)
        sns.heatmap(pic, cmap=cmap, norm=norm)
        plt.xticks([])
        plt.yticks([])
        plt.title(f'Scale= {closest_number*1.5/200:.2f}, N='+str(max(Community) + 1),fontsize=20)
        plt.tight_layout()
        #plt.show()
        if env == 'fam1':
            plt.savefig('/Users/sonmjack/Downloads/age2 result_fam1/Signal_Markov_fig_AD/'+str(index)+'_young_fam1_markov_neuron_mask.pdf')
            plt.close()

        elif env =="nov":
            plt.savefig('/Users/sonmjack/Downloads/age2 result_nov/Signal_Markov_fig_AD/'+str(index)+'_young_nov_markov_neuron_mask.pdf')
            plt.close()
        elif env =="fam1r2":
            plt.savefig('/Users/sonmjack/Downloads/age2 result_fam1r2/Signal_Markov_fig_AD/mask_'+str(index)+'_young_fam1r2_markov_neuron.pdf')
            plt.close()            

        
        if env == 'fam1':
            name = '/Users/sonmjack/Downloads/age2 result_fam1/Signal_Markov_fig_AD/'+str(index)+'_young_fam1_sankey.png'   
        elif env =="nov":
            name = '/Users/sonmjack/Downloads/age2 result_nov/Signal_Markov_fig_AD/'+str(index)+'_young_nov_snakey.png'
        elif env =="fam1r2":
            name = '/Users/sonmjack/Downloads/age2 result_fam1r2/Signal_Markov_fig_AD/snakey'+str(index)+'_young_fam1r2.png'
        
            
        plot_sankey(
            all_results,
            optimal_scales=True,
            live=False,
            filename=name ,
            scale_index=None,
        )

        v = v+1

/var/folders/jz/rt8774f93qn00g88dj6zpmn40000gn/T/ipykernel_29720/526107217.py:29: UserWarning:

This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.

/var/folders/jz/rt8774f93qn00g88dj6zpmn40000gn/T/ipykernel_29720/526107217.py:105: UserWarning:

FixedFormatter should only be used together with FixedLocator

/var/folders/jz/rt8774f93qn00g88dj6zpmn40000gn/T/ipykernel_29720/526107217.py:29: UserWarning:

This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.

/var/folders/jz/rt8774f93qn00g88dj6zpmn40000gn/T/ipykernel_29720/526107217.py:105: UserWarning:

FixedFormatter should only be used together with FixedLocator

/var/folders/jz/rt8774f93qn00g88dj6zpmn40000gn/T/ipykernel_29720/526107217.py:267: UserWarning:

This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.

/var/folders/jz/rt8774f93qn00g88dj6zpmn40000gn/T/ipykernel_29720/526107217.py:342: UserW

## Community based on EPSP network

In [43]:
for index in range(len(dy_list)):
    if  gene_list[index] == 119:
        type = 'wild type'
        A = dy_list[index]
        A = normal(A)
        Q = sparse(A)
        adjacency = sp.csr_matrix(Q)

        all_results = pgs.run(adjacency,
                              min_scale=-1.5,
                              max_scale=1.5,
                              n_scale=200,
                              n_tries = 600,
                              constructor="directed")

        all_results = pgs.identify_optimal_scales(all_results, kernel_size=14, window_size=14)
        
        if env == 'fam1':
            with open('/Users/sonmjack/Downloads/age2 result_fam1/fam1_S_Markov_'+ str(index) +'.pkl', 'wb') as f:
                pickle.dump(all_results, f)
        elif env =="nov":
            with open('/Users/sonmjack/Downloads/age2 result_nov/nov_S_Markov_'+ str(index) +'.pkl', 'wb') as f:
                pickle.dump(all_results, f)
        elif env=='fam1r2':
            with open('/Users/sonmjack/Downloads/age2 result_fam1r2/fam1r2_S_Markov_'+ str(index) +'.pkl', 'wb') as f:
                pickle.dump(all_results, f)

    else: pass

Your graph is directed!
100%|██████████| 200/200 [00:10<00:00, 19.55it/s]
Your graph is directed!
100%|██████████| 200/200 [00:14<00:00, 13.72it/s]
Your graph is directed!
100%|██████████| 200/200 [00:02<00:00, 97.29it/s] 
Your graph is directed!
100%|██████████| 200/200 [00:02<00:00, 70.58it/s]
Your graph is directed!
100%|██████████| 200/200 [00:05<00:00, 36.64it/s]
Your graph is directed!
100%|██████████| 200/200 [00:07<00:00, 27.48it/s]
Your graph is directed!
100%|██████████| 200/200 [00:06<00:00, 32.05it/s]
Your graph is directed!
100%|██████████| 200/200 [00:06<00:00, 31.25it/s]
Your graph is directed!
100%|██████████| 200/200 [00:03<00:00, 53.79it/s]
Your graph is directed!
100%|██████████| 200/200 [00:16<00:00, 11.79it/s]
Your graph is directed!
100%|██████████| 200/200 [00:02<00:00, 84.54it/s]


## (EPSP) Analysis of community and mask

In [23]:
import numpy as np
import matplotlib.colors as mcolors
import random

cluster_number = []

Color_Code2 = ['white','darkblue','red', "orange", "pink", "olive", "cyan","yellow", "green", "brown", "gray", "tomato",'limegreen']

list_color = []
for color in mcolors.CSS4_COLORS:
    list_color.append(color)
random.shuffle(list_color)
Color_Code1 = [*Color_Code2, *list_color]
Color_Code = [*Color_Code1, *list_color]
        
for index in range(len(dy_list)):#len(dy_list)
    plt.close()
    if gene_list[index] == 119:
        Spike_train = neuron_spike[index]
        Spike_train = Spike_train[:,0:int(Spike_train.shape[1]/3)]
        mask_index = mask[index]
        tuning_curve = All_tuning_curve[index]
        type = 'wild type'
        if env == 'fam1':
            with open('/Users/sonmjack/Downloads/age2 result_fam1/fam1_S_Markov_'+ str(index) +'.pkl', 'rb') as file:
                all_results = pickle.load(file)
            _ = pgs.plot_scan(all_results)
            plt.savefig('/Users/sonmjack/Downloads/age2 result_fam1/Markov_fig/'+str(index)+'_young_fam1_markov.png')
            plt.close()

        elif env =="nov":
            with open('/Users/sonmjack/Downloads/age2 result_nov/nov_S_Markov_'+ str(index) +'.pkl', 'rb') as file:
                all_results = pickle.load(file)
            _ = pgs.plot_scan(all_results)
            plt.savefig('/Users/sonmjack/Downloads/age2 result_nov/Markov_fig/'+str(index)+'_young_nov_markov.png')
            plt.close()
        elif env =="fam1r2":
            with open('/Users/sonmjack/Downloads/age2 result_fam1r2/fam1r2_S_Markov_'+ str(index) +'.pkl', 'rb') as file:
                all_results = pickle.load(file)
            _ = pgs.plot_scan(all_results)
            plt.savefig('/Users/sonmjack/Downloads/age2 result_fam1r2/Markov_fig/'+str(index)+'_young_fam1r2_markov.png')
            plt.close()
            
        if env == 'fam1'or env=='fam1r2':
            closest_number = min(all_results['selected_partitions'], key=lambda x: abs(x - 133))
            Community = all_results['community_id'][closest_number]
        elif env =="nov" :
            selected_partitions = all_results['selected_partitions']
            Community = all_results['community_id'][199]
        
        
        non_zero_indices_per_row = []
        for row in Spike_train:
            # 找到每行中不为0的元素的列索引
            non_zero_indices = np.where(row != 0)[0]
            # 添加到列表中
            non_zero_indices_per_row.append(list(non_zero_indices))
        neuralData = []
        ColorCode = []
        C_tuning_curve = []
        
        for i in range(max(Community) + 1):
            for m in range(len(non_zero_indices_per_row)):
                if Community[m] == i:
                    C_tuning_curve.append(tuning_curve[m])
                if m == len(non_zero_indices_per_row)-1:
                    C_tuning_curve.append(np.full(tuning_curve[m].shape[0], np.nan))
                    C_tuning_curve.append(np.full(tuning_curve[m].shape[0], np.nan))
                    C_tuning_curve.append(np.full(tuning_curve[m].shape[0], np.nan))
        
        C_tuning_curve = np.array(C_tuning_curve)
        plt.close()
        plt.figure(figsize=(10, 6))
        sns.heatmap(C_tuning_curve, cmap='viridis',vmax=1)
        
        if env == 'fam1':
            plt.savefig('/Users/sonmjack/Downloads/age2 result_fam1/Markov_fig/'+str(index)+'_young_fam1_tuning.png')
            plt.close()    
        elif env =="nov":
            plt.savefig('/Users/sonmjack/Downloads/age2 result_nov/Markov_fig/'+str(index)+'_young_nov_tuning.png')
            plt.close()
        elif env =="fam1r2":
            plt.savefig('/Users/sonmjack/Downloads/age2 result_fam1r2/Markov_fig/'+str(index)+'_young_fam1r2_tuning.png')
            plt.close()
        
        
        
        it = 0
        Nodes = range(len(non_zero_indices_per_row))
        for m, j in zip(Nodes, Nodes):
            for i in range(max(Community) + 1):
                if Community[j] == i:
                    non_zero_indices_per_row[int(m)].sort()
                    neuralData.append(non_zero_indices_per_row[int(m)])
                    ColorCode.append(Color_Code[i+1])
                    # A_ordered_row[it,:]=Q[j,:]
                    it += 1
        fig, axes = plt.subplots(2, 1, figsize=(27, 9), gridspec_kw={'height_ratios': [6, 3]})
        axes[0].eventplot(neuralData, color=ColorCode)
        axes[0].invert_yaxis()

        axes[0].set_title("Reordered neural spike trains " + f'N-{max(Community) + 1}')
        axes[0].set_xlabel('Time', fontsize=13)
        axes[0].set_ylabel('Rearranged Neuron ID', fontsize=13)
        
       # max_in_sublists = [max(sublist) for sublist in neuralData if sublist]
        
        # 找到最大值中的最大值
        #overall_max = max(max_in_sublists)
        #nearest_5000_multiple = round(overall_max / 10000) * 10000
        #axes[0].set_xticks(np.arange(0,nearest_5000_multiple, 10000), [str(int(i*10000/30)) for i in range(0, round(overall_max / 10000))],fontsize=13)
        #plt.show()
        

        
        if env == 'fam1':
            axes[1].plot(be_phi_list_young_fam1[index][0:int(neuron_spike[index].shape[1]/3)])
            #axes[1].set_xticks(np.arange(0,nearest_5000_multiple, 10000), [str(int(i*10000/30)) for i in range(0, round(overall_max / 10000))],fontsize=13)
            axes[1].invert_yaxis()
            plt.savefig('/Users/sonmjack/Downloads/age2 result_fam1/Markov_fig/'+str(index)+'_young_fam1_markov_neuron_list.png')
            plt.close()    
        elif env =="nov":
            axes[1].plot(be_phi_list_young_nov[index][0:int(neuron_spike[index].shape[1]/3)])
            #axes[1].set_xticks(np.arange(0,nearest_5000_multiple, 10000), [str(int(i*10000/30)) for i in range(0, round(overall_max / 10000))],fontsize=13) 
            axes[1].invert_yaxis()
            plt.savefig('/Users/sonmjack/Downloads/age2 result_nov/Markov_fig/'+str(index)+'_young_nov_markov_neuron_list.png')
            plt.close()
        elif env =="fam1r2":
            axes[1].plot(be_phi_list_young_fam1r2[index][0:int(neuron_spike[index].shape[1]/3)])
            #axes[1].set_xticks(np.arange(0,nearest_5000_multiple, 10000), [str(int(i*10000/30)) for i in range(0, round(overall_max / 10000))],fontsize=13)   
            axes[1].invert_yaxis()
            plt.savefig('/Users/sonmjack/Downloads/age2 result_fam1r2/Markov_fig/neuron_list_'+str(index)+'_young_fam1r2_markov.png')
            plt.close()
            
            
        from matplotlib.colors import BoundaryNorm
        
        pic = np.zeros((512, 512))
        for i in range(mask_index.shape[2]):
            index_row = np.where(mask_index[:, :, i] == True)[0]
            index_con = np.where(mask_index[:, :, i] == True)[1]
            for j in range(len(index_row)):
                    pic[index_row[j], index_con[j]] = Community[i]+1
        #cmap = sns.diverging_palette(220, 20, as_cmap=True)
        import seaborn as sns
        
        cmap = plt.get_cmap('tab10')  

        from matplotlib.colors import LinearSegmentedColormap
        cmap = LinearSegmentedColormap.from_list("custom_cmap", Color_Code2, N=len(Color_Code2))
        bounds = np.linspace(-0.5, 11.5, 13)  
        norm = BoundaryNorm(bounds, cmap.N)
        sns.heatmap(pic, cmap=cmap, norm=norm)
        
        plt.title('Community beasd on real data')        
        
        if env == 'fam1':
            plt.savefig('/Users/sonmjack/Downloads/age2 result_fam1/Markov_fig/'+str(index)+'_young_fam1_markov_neuron_mask.png')
            plt.close()
            
        elif env =="nov":
            plt.savefig('/Users/sonmjack/Downloads/age2 result_nov/Markov_fig/'+str(index)+'_young_nov_markov_neuron_mask.png')
            plt.close()
        elif env =="fam1r2":
            plt.savefig('/Users/sonmjack/Downloads/age2 result_fam1r2/Markov_fig/mask_'+str(index)+'_young_fam1r2_markov_neuron.png')
            plt.close()  


        if env == 'fam1':
            name = '/Users/sonmjack/Downloads/age2 result_fam1/Markov_fig/'+str(index)+'_young_fam1_sankey.png'   
        elif env =="nov":
            name = '/Users/sonmjack/Downloads/age2 result_nov/Markov_fig/'+str(index)+'_young_nov_snakey.png'
        elif env =="fam1r2":
            name = '/Users/sonmjack/Downloads/age2 result_fam1r2/Markov_fig/snakey'+str(index)+'_young_fam1r2.png'
            
            
        plot_sankey(
            all_results,
            optimal_scales=True,
            live=False,
            filename=name ,
            scale_index=None,
        )

In [45]:
# def pre_process3(data):
#     dy_r = np.array(data)
#     t_p = np.ravel(dy_r.copy())
#     return t_p
# be_g1 = pre_process3(be_g1)
# be_g2 = pre_process3(be_g2)
# be_g3 = pre_process3(be_g3)
# be_g4 = pre_process3(be_g4)
# 
# plt.figure(figsize=(8, 6))
# bins = np.linspace(0, 360, num=36)
# sns.histplot(be_g1, color='red', kde=True, label='Community 1', alpha= 0.5, bins=bins)
# sns.histplot(be_g2, color='green', kde=True, label='Community 2', alpha= 0.5,bins=bins)
# sns.histplot(be_g3, color='blue', kde=True, label='Community 3', alpha= 0.5,bins=bins)
# sns.histplot(be_g4, color='skyblue', kde=True, label='Community 4', alpha= 0.5, bins=bins)
# 
# plt.axvline(np.mean(be_g1), color='red', linestyle='--', label='Mean of Community 1')
# plt.axvline(np.mean(be_g2), color='green', linestyle='--', label='Mean of Community 2')
# plt.axvline(np.mean(be_g3), color='blue', linestyle='--', label='Mean of Community 3')
# plt.axvline(np.mean(be_g4), color='skyblue', linestyle='--', label='Mean of Community 4')
# 
# # 添加图例
# plt.legend()
# plt.xlabel("Angle on the circular track")
# # 显示图形
# plt.show()